In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import time

In [4]:
csv_file = tf.keras.utils.get_file("heart.csv", "https://storage.googleapis.com/download.tensorflow.org/data/heart.csv")

df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [7]:
df.thal = pd.Categorical(df.thal)
df.thal = df.thal.cat.codes

In [8]:
train_dataset = df.sample(frac = 0.8, random_state = 0)
test_dataset = df.drop(train_dataset.index)

train_labels = train_dataset.pop("target")  # does this in place 
test_labels = test_dataset.pop("target")

In [9]:
def norm(x, train_stats):
    # scale data, x = dataset to scale, train_stats is description of dataset
    return(x - train_stats["mean"]) / train_stats["std"]

In [11]:
train_stats = train_dataset.describe().T

normed_train_data = norm(train_dataset, train_stats)

normed_train_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
225,0.351009,-1.420082,0.89547,-0.115110,-0.934323,-0.389553,-1.035919,-0.778745,-0.668765,-0.391781,0.687374,-0.744473,-0.444130
152,-1.073860,0.701275,0.89547,0.546830,1.213225,-0.389553,0.985794,-0.087162,1.489116,-0.919976,0.687374,2.434865,1.113543
228,0.022193,-1.420082,0.89547,-0.225433,-0.780926,-0.389553,-0.025063,-0.821968,1.489116,0.840674,0.687374,0.315306,1.113543
201,-1.293070,0.701275,0.89547,-0.942535,1.098178,-0.389553,-1.035919,1.382450,-0.668765,0.136414,0.687374,-0.744473,-0.444130
52,-0.197018,-1.420082,0.89547,-0.115110,0.350371,-0.389553,0.985794,-0.260058,-0.668765,-0.567846,0.687374,-0.744473,-0.444130


In [13]:
test_stats = test_dataset.describe().T

normed_test_data = norm(test_dataset, test_stats)

In [15]:
model = keras.Sequential([
    # input shape is len of col of train data in 1d array form
    keras.layers.Dense(64, activation ="relu", input_shape = [len(normed_train_data.keys())]),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.Dense(1)
])

optimizer = keras.optimizers.RMSprop(0.001)
model.compile(loss = "mse", optimizer = optimizer,
             metrics = ["mse", "mae"])

In [16]:
history = model.fit(normed_train_data, train_labels, epochs = 10)

Epoch 1/10
8/8 [==============================] - 0s 3ms/step - loss: 0.2800 - mse: 0.2800 - mae: 0.3795
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 0.1400 - mse: 0.1400 - mae: 0.2902
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 0.1183 - mse: 0.1183 - mae: 0.2568
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1067 - mse: 0.1067 - mae: 0.2354
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 0.0972 - mse: 0.0972 - mae: 0.2262
Epoch 6/10
8/8 [==============================] - 0s 3ms/step - loss: 0.0910 - mse: 0.0910 - mae: 0.2173
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0844 - mse: 0.0844 - mae: 0.2089
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0768 - mse: 0.0768 - mae: 0.1956
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0728 - mse: 0.0728 - mae: 0.1958
Epoch 10/10
8/8 [==============================] - 0s 2

In [27]:
y_pred = model.predict(normed_test_data)

In [28]:
# y_pred = [x[0] for x in y_pred]

keras.losses.MAE(test_labels.values, y_pred)

<tf.Tensor: shape=(61,), dtype=float32, numpy=
array([0.60027325, 0.6386521 , 0.30599478, 0.3189914 , 0.29377082,
       0.4617992 , 0.37352636, 0.28545448, 0.5193871 , 0.4267507 ,
       0.41621056, 0.57659566, 0.31448594, 0.3102503 , 0.36038443,
       0.67665935, 0.78468776, 0.33418894, 0.6072789 , 0.4323941 ,
       0.81607646, 0.47127092, 0.43307233, 0.30169943, 0.409288  ,
       0.43157265, 0.36134693, 0.30971983, 0.3011577 , 0.32135937,
       0.281767  , 0.44444364, 0.29489473, 0.29056332, 0.3385855 ,
       0.46793053, 0.61990017, 0.35353422, 0.33451897, 0.35336122,
       0.31072053, 0.3656328 , 0.35109276, 0.33535376, 0.46225336,
       0.4721593 , 0.36538088, 0.3686503 , 0.49647138, 0.28912616,
       0.5095542 , 0.30722314, 0.5790323 , 0.28602386, 0.40134925,
       0.6278302 , 0.36732483, 0.66884536, 0.54064155, 0.3075116 ,
       0.39769238], dtype=float32)>

In [29]:
model.evaluate(normed_test_data, test_labels)

2/2 [==============================] - 0s 4ms/step - loss: 0.1618 - mse: 0.1618 - mae: 0.2998


[0.16178062558174133, 0.16178062558174133, 0.29982826113700867]

**Batching and Prefetching**

In [34]:
model = keras.Sequential([
    # input shape is len of col of train data in 1d array form
    keras.layers.Dense(64, activation ="relu", input_shape = [len(normed_train_data.keys())]),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.Dense(1)
])

optimizer = keras.optimizers.RMSprop(0.001)
model.compile(loss = "mse", optimizer = optimizer,
             metrics = ["mse", "mae"])

In [35]:
# batching
dataset = tf.data.Dataset.from_tensor_slices(
    (normed_train_data.values, train_labels.values)
).batch(10)

history = model.fit(dataset, epochs = 100)

Epoch 1/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1811 - mse: 0.1811 - mae: 0.3299
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1089 - mse: 0.1089 - mae: 0.2470
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0934 - mse: 0.0934 - mae: 0.2221
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0823 - mse: 0.0823 - mae: 0.2037
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.0728 - mse: 0.0728 - mae: 0.1887
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0649 - mse: 0.0649 - mae: 0.1762
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0582 - mse: 0.0582 - mae: 0.1651
Epoch 8/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0523 - mse: 0.0523 - mae: 0.1545
Epoch 9/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0470 - mse: 0.0470 - mae: 0.1450
Epoch 10/100
25/25 [========

In [36]:
model = keras.Sequential([
    # input shape is len of col of train data in 1d array form
    keras.layers.Dense(64, activation ="relu", input_shape = [len(normed_train_data.keys())]),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.Dense(1)
])

optimizer = keras.optimizers.RMSprop(0.001)
model.compile(loss = "mse", optimizer = optimizer,
             metrics = ["mse", "mae"])

In [37]:
# prefetching
dataset = tf.data.Dataset.from_tensor_slices(
    (normed_train_data.values, train_labels.values)
).batch(10).prefetch(2)  # prefetches 2 batches at a time

history = model.fit(dataset, epochs = 100)

Epoch 1/100
25/25 [==============================] - 0s 2ms/step - loss: 0.1906 - mse: 0.1906 - mae: 0.3201
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 0.1174 - mse: 0.1174 - mae: 0.2507
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.1012 - mse: 0.1012 - mae: 0.2305
Epoch 4/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0894 - mse: 0.0894 - mae: 0.2154
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0803 - mse: 0.0803 - mae: 0.2030
Epoch 6/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0732 - mse: 0.0732 - mae: 0.1925
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0671 - mse: 0.0671 - mae: 0.1826
Epoch 8/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0616 - mse: 0.0616 - mae: 0.1728
Epoch 9/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0568 - mse: 0.0568 - mae: 0.1642
Epoch 10/100
25/25 [========

**Parallelizing data extraction**

In [38]:
# mostly used when needed data is stored remotely
tf.data.Dataset.interleave(
    dataset,
    num_parallel_calls=tf.data.experimental.AUTOTUNE  # auto decide optimal num threads to use
)

TypeError: interleave() missing 1 required positional argument: 'map_func'